# This notebook is used to do custom voice TTS using YourTTS

#### Download and install Coqui as per yourtts_readme.txt

complete steps 1 and 2 of the readme

In [8]:
!pip install -q torchaudio ipywidgets

In [5]:
!tts --list_models

 Name format: type/language/dataset/model
 1: tts_models/multilingual/multi-dataset/your_tts
 2: tts_models/en/ek1/tacotron2
 3: tts_models/en/ljspeech/tacotron2-DDC [already downloaded]
 4: tts_models/en/ljspeech/tacotron2-DDC_ph
 5: tts_models/en/ljspeech/glow-tts [already downloaded]
 6: tts_models/en/ljspeech/speedy-speech
 7: tts_models/en/ljspeech/tacotron2-DCA
 8: tts_models/en/ljspeech/vits [already downloaded]
 9: tts_models/en/ljspeech/fast_pitch [already downloaded]
 10: tts_models/en/vctk/vits [already downloaded]
 11: tts_models/en/vctk/fast_pitch
 12: tts_models/en/sam/tacotron-DDC
 13: tts_models/en/blizzard2013/capacitron-t2-c50
 14: tts_models/en/blizzard2013/capacitron-t2-c150
 15: tts_models/es/mai/tacotron2-DDC
 16: tts_models/fr/mai/tacotron2-DDC
 17: tts_models/uk/mai/glow-tts
 18: tts_models/zh-CN/baker/tacotron2-DDC-GST
 19: tts_models/nl/mai/tacotron2-DDC
 20: tts_models/de/thorsten/tacotron2-DCA
 21: tts_models/de/thorsten/vits
 22: tts_models/ja/kokoro/tacotr

Get the YourTTS model and configs


In [6]:
!wget https://coqui.gateway.scarf.sh/v0.6.1_models/tts_models--multilingual--multi-dataset--your_tts.zip

--2022-09-19 17:06:45--  https://coqui.gateway.scarf.sh/v0.6.1_models/tts_models--multilingual--multi-dataset--your_tts.zip
Resolving coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)... 35.155.221.103, 54.70.21.136
Connecting to coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)|35.155.221.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/coqui-ai/TTS/releases/download/v0.6.1_models/tts_models--multilingual--multi-dataset--your_tts.zip [following]
--2022-09-19 17:06:45--  https://github.com/coqui-ai/TTS/releases/download/v0.6.1_models/tts_models--multilingual--multi-dataset--your_tts.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/265612440/9d9fb558-77b2-4a66-b02e-61f7aaf790bb?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential

Follow Step 3 in the readme

Imports

In [9]:
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio

from glob import glob

import torch

from TTS.tts.utils.synthesis import synthesis

# This line was picked from https://colab.research.google.com/drive/1r0NDBxxW5RZjQ1Jy99XohnY6thYWNBCd?usp=sharing#scrollTo=2akFqoi7UiD4
# This import was not resolved
# from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
from TTS.tts.utils.text.tokenizer import TTSTokenizer
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *

Paths for model and config downloaded (yourtts_models)

In [10]:
OUT_PATH = 'yourtts_out/'

# create output path
os.makedirs(OUT_PATH, exist_ok=True)

# model vars 
MODEL_PATH = './yourtts_models/model_file.pth'
CONFIG_PATH = './yourtts_models/config.json'

TTS_SPEAKERS = "./yourtts_models/speakers.json"
# SE = Speaker Encoder
SE_MODEL_PATH="./yourtts_models/model_se.pth"
CONFIG_SE_PATH = "./yourtts_models/config_se.json"
print(CONFIG_PATH)
USE_CUDA = torch.cuda.is_available()

./yourtts_models/config.json


Restore model

In [11]:
# load the config, config path was defined above
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False
C.model_args['speaker_encoder_config_path'] = CONFIG_SE_PATH
C.model_args['speaker_encoder_model_path'] = SE_MODEL_PATH

model = setup_model(C)
# model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)
# print(model.language_manager.num_languages, model.embedded_language_dim)
# print(model.emb_l)
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))



 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:

This is important - remove the speaker encoder

In [12]:

# remove speaker encoder
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  if "speaker_encoder" in key:
    del model_weights[key]

model.load_state_dict(model_weights)


model.eval()

if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

#### Speaker Encoder Setup

install helper libraries

In [13]:
! pip install -q pydub ffmpeg-normalize==1.21.0

Imports

In [14]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
import librosa

load the Speaker Encoder

In [15]:
SE_speaker_manager = SpeakerManager(encoder_model_path=SE_MODEL_PATH, \
                                    encoder_config_path=CONFIG_SE_PATH, \
                                        use_cuda=USE_CUDA)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:True
 | > db_level:-27.0
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400


Define a helper function

In [16]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

#### Voice Conversion

##### Upload, normalize and resample your wav files

Upload wav files - I have saved the wavs in Libri-speech format - directory MyTTSDataset/en

In [17]:
WAVS_PATH = "./MyTTSDataset/en/wavs/"
reference_files = os.listdir(WAVS_PATH)
new_list = [ WAVS_PATH+s for s in reference_files]

#print(new_list)
for sample in new_list:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f
    

Compute Embedding

In [18]:
reference_emb = SE_speaker_manager.compute_embedding_from_clip(new_list)

In [19]:
print(reference_emb)

[0.010579373687505722, 0.013064173050224781, 0.0012921703746542335, -0.019454441964626312, 0.006346800364553928, 0.020704161375761032, -0.014509971253573895, -0.0026169302873313427, -0.0009212891454808414, 0.00535170640796423, -0.002093872055411339, 0.003846731735393405, -0.02831941470503807, 0.026533663272857666, -0.03715554252266884, 0.05479437857866287, 0.09971534460783005, 0.06859470903873444, 0.0030610316898673773, 0.026424290612339973, -0.022029325366020203, 0.04718051850795746, 0.02066671848297119, 0.02186550386250019, 0.011537215672433376, 0.006842123810201883, -0.0024214177392423153, -0.020184321328997612, -0.03675965592265129, 0.003910921514034271, 0.024177970364689827, -0.0019377605058252811, 0.013028427958488464, 0.017538921907544136, 0.004976797383278608, 0.009017490781843662, -0.02612612396478653, -0.008424101397395134, -0.0106161804869771, 0.0012517039431259036, 0.005137331783771515, 0.007476564031094313, -0.004499954637140036, 0.015952087938785553, -0.009027846157550812

define inference variables

In [24]:
model.length_scale = 1  # scaler for the duration predictor. The larger it is, the slower the speech.
model.inference_noise_scale = 0.3 # defines the noise variance applied to the random z vector at inference.
model.inference_noise_scale_dp = 0.3 # defines the noise variance applied to the duration predictor z vector at inference.
text = "It took me quite a long time to develop a voice and now that I have it I am not going to be silent."

In [21]:
#  Choose language id
model.language_manager.ids

{'en': 0, 'fr-fr': 1, 'pt-br': 2}

In [22]:
language_id = 0 # english

#### Synthesis

In [25]:
from typing import Any


print(" > text: {}".format(text))
wav, alignment, _, _ = synthesis(
                    model,
                    text,
                    C,
                    use_cuda = USE_CUDA,
                    speaker_id=None,
                    style_wav=None,
                    use_griffin_lim=True,
                    do_trim_silence=False,
                    d_vector=reference_emb,
                    language_id=language_id,
                ).values()
print("Generated Audio")
IPython.display.display(Audio(wav, rate=ap.sample_rate))
file_name = "text1.wav"
#file_name = file_name.translate(str.maketrans('', '', string.punctuation.replace('_', ''))) + '.wav'
out_path = os.path.join(OUT_PATH, file_name)
print(" > Saving output to {}".format(out_path))
ap.save_wav(wav, out_path)

 > text: It took me quite a long time to develop a voice and now that I have it I am not going to be silent.
Generated Audio


 > Saving output to yourtts_out/text1.wav
